In [1]:
param_date = '20240623'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/2.subgroupname.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Tab_No.'] = input_['Tab_No.'].astype(int)
input_['GroupName_No.'] = input_['GroupName_No.'].astype(int)
input_['SubGroupName_No.'] = input_['SubGroupName_No.'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'navnode_fetch',
                        'payload': f'''{{"jsn":{{"tab":"extras","groupname":"{input_.loc[a, 'GroupName']}","subgroupname":"{input_.loc[a, 'SubGroupName']}","nodetype":"subgroupname"}}}}''',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp)

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_json = html.xpath('//input[contains(@id, "jsn[")]/@value')

                # = = = = = = = = = = = = = = =

                list_parttype = [json.loads(json_)['parttype'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_a = html.xpath('//td[@class="nlabel"]/a')

                # = = = = = = = = = = = = = = =

                list_a = [re.sub(r' +', ' ', a.text.strip()) for a in list_a]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Tab_No.': input_.loc[a, 'Tab_No.'],
                                        'Tab': input_.loc[a, 'Tab'],
                                        'GroupName_No.': input_.loc[a, 'GroupName_No.'],
                                        'GroupName': input_.loc[a, 'GroupName'],
                                        'SubGroupName_No.': input_.loc[a, 'SubGroupName_No.'],
                                        'SubGroupName': input_.loc[a, 'SubGroupName'],
                                        'PartType_No.': [i+1 for i in range(len(list_parttype))],
                                        'PartType_Name': list_a,
                                        'PartType': list_parttype,
                                        'Tab_Url': [f'''https://www.rockauto.com/en/tools/{input_.loc[a, 'GroupName']},{input_.loc[a, 'SubGroupName']},{parttype_name},{parttype}''' for parttype_name, parttype in zip(list_a, list_parttype)]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Tab_No.': input_.loc[a, 'Tab_No.'],
                                     'Tab': input_.loc[a, 'Tab'],
                                     'GroupName_No.': input_.loc[a, 'GroupName_No.'],
                                     'GroupName': input_.loc[a, 'GroupName'],
                                     'SubGroupName_No.': input_.loc[a, 'SubGroupName_No.'],
                                     'SubGroupName': input_.loc[a, 'SubGroupName']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'GroupName_No.']}. {input_.loc[a, 'GroupName']} > {input_.loc[a, 'SubGroupName_No.']}. {input_.loc[a, 'SubGroupName']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['Tab_No.', 'GroupName_No.', 'SubGroupName_No.', 'PartType_No.'], ascending=[True, True, True, True], ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/3.parttype-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['Tab_No.', 'GroupName_No.', 'SubGroupName_No.'], ascending=[True, True, True], ignore_index=True)
    output_error.to_excel(f'./{param_date}/parttype_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：331

[状态：ok，尝试次数：  1] - 3. Body & Lamp Assembly > 1. Accessory Lighting
[剩余数量：311] - [当前时间：10:49:15]

[状态：ok，尝试次数：  1] - 1. Apparel & Gifts > 1. RockAuto Apparel
[剩余数量：310] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 6. Pulley & Tensioner
[剩余数量：309] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 3. Body & Lamp Assembly > 5. Bumper / Pad
[剩余数量：308] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 1. Apparel & Gifts > 2. RockAuto Gift
[剩余数量：307] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 3. Body & Lamp Assembly > 9. Fender
[剩余数量：306] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt
[剩余数量：305] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 3. Body & Lamp Assembly > 6. Cargo Management
[剩余数量：304] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 3. Body & Lamp Assembly > 4. Body Panel
[剩余数量：303] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 3. Body & Lamp Assembly > 3. Back Up / Reverse Lamp
[剩余数量：302] - [当前时间：10:49:16]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 2. Automotive V-Belt
[剩余数量：301] - [当前时间：10:49:17]

